## Install/Import Dependencies

In [19]:
%pip install spacy geopy folium

In [ ]:
# Cell 1: Imports & API key
import pandas as pd
import spacy
from geopy.geocoders import Nominatim
import time
import folium


## Load Data

In [21]:

tweets_df = pd.read_csv('annapolis_flood_tweets_expanded_sample.csv')
tweets_df.head()


tweet_id                                         tweet_text   latitude  \
0         1  Flooding reported near City Dock, water levels...  38.974997   
1         2  Streets around Eastport are underwater after h...  38.973759   
2         3  High tides causing floods again downtown Annap...  38.983738   
3         4  Flash flooding near Forest Drive, avoid the area!  38.975188   
4         5  Minor flooding noticed at Quiet Waters Park en...  38.969106   

   longitude        date severity  
0 -76.488239  2024-05-03   Medium  
1 -76.495053  2024-05-03     High  
2 -76.490363  2024-05-03     High  
3 -76.494183  2024-05-03     High  
4 -76.496130  2024-05-03     High

## Load spaCy and extract toponyms

In [29]:

# !python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')

def extract_toponym(text):
    doc = nlp(text)
    # keep GPE or LOC entities
    ents = [ent.text for ent in doc.ents if ent.label_ in ('GPE','LOC')]
    return ents[0] if ents else None

tweets_df['toponym'] = tweets_df['tweet_text'].apply(extract_toponym)
tweets_df[['tweet_text','toponym']].head(50)


tweet_text  \
0   Flooding reported near City Dock, water levels...   
1   Streets around Eastport are underwater after h...   
2   High tides causing floods again downtown Annap...   
3   Flash flooding near Forest Drive, avoid the area!   
4   Minor flooding noticed at Quiet Waters Park en...   
5   Massive puddles forming near Naval Academy due...   
6   Storm drains overwhelmed near West Street, flo...   
7   High water blocking roads near Spa Creek, cars...   
8   Residential flooding observed near Bay Ridge, ...   
9   Water rapidly rising near Truxtun Park after r...   
10      Floodwaters impacting traffic near Rowe Blvd.   
11  Dock Street under significant flooding after s...   
12    Neighborhood streets near Back Creek submerged.   
13  Severn Avenue experiencing floods, residents c...   
14   Chesapeake Avenue closed due to severe flooding.   
15       Flooding observed around King George Street.   
16  Heavy flooding on Compromise Street, caution a...   
17  Streets near Jonas Green Park flooded after re...   
18    Rising waters causing issues near Maryland Ave.   
19  Flood warning near Eastport Bridge, dangerous ...   
20  Flood conditions worsening near Main Street, w...   
21  Parking lots near Calvert Street flooded, cars...   
22  Flooding disrupts business at Market Space dow...   
23  Storm surge causing floods near Annapolis Harbor.   
24  Reports of basement flooding in homes around C...   
25  Flooded sidewalks reported near St. John's Col...   
26  Severe flooding observed at Acton's Landing Park.   
27  Water levels rising near Ego Alley, significan...   
28  Residents report backyard flooding near Hillto...   
29  Standing water causing traffic delays on Duke ...   
30  Flood warnings issued for areas near College C...   
31  Water overflow near Amos Garrett Blvd, road cl...   
32  Flooding near Germantown Elementary School aft...   
33  Flooded conditions around Bestgate Road, drive...   
34  Extreme flooding near Anne Arundel Medical Cen...   
35   City Dock submerged again after recent rainfall.   
36  Side streets near President St flooded due to ...   
37  Critical flooding conditions near Annapolis Ma...   
38  Storm causing flooding around Spa Road, cautio...   
39  Neighborhood streets around Tyler Avenue exper...   
40  Reports of flood damage around West Annapolis ...   
41  Heavy flooding affecting homes near Admiral He...   
42  Businesses around Taylor Avenue facing floodin...   
43  Flooding severe near Weems Creek, residents ev...   
44  Water rescue operations near Broad Creek due t...   
45  Rising waters observed near historic district ...   
46    Minor street flooding near Annapolis Mall area.   
47  Floodwaters blocking access to Quiet Waters Do...   
48  Emergency services dealing with floods near Ad...   
49  Reports of flooded streets near the State Capi...   

                      toponym  
0                        None  
1                    Eastport  
2                        None  
3                        None  
4                 Waters Park  
5                        None  
6                 West Street  
7                   Spa Creek  
8                   Bay Ridge  
9                        None  
10                       None  
11                       None  
12                 Back Creek  
13              Severn Avenue  
14                       None  
15                       None  
16                       None  
17                       None  
18                   Maryland  
19                       None  
20                       None  
21                       None  
22                       None  
23           Annapolis Harbor  
24                       None  
25         St. John's College  
26                      Acton  
27                       None  
28                       None  
29                       None  
30              College Creek  
31                       None  
32                       None  
33                       None  
34             

## Geocode extracted toponyms

In [30]:
# Cell 4: Geocode extracted toponyms

geolocator = Nominatim(user_agent='lab4_geocoder')

def geocode_place(place):
    if not place:
        return None, None
    try:
        loc = geolocator.geocode(f"{place}, Annapolis, MD")
        time.sleep(1)  # rate-limit
        return (loc.latitude, loc.longitude) if loc else (None, None)
    except:
        return (None, None)

coords = tweets_df['toponym'].apply(geocode_place)
tweets_df['pred_lat'], tweets_df['pred_lon'] = zip(*coords)
tweets_df[['toponym','pred_lat','pred_lon']].head(50)


toponym   pred_lat   pred_lon
0                        None        NaN        NaN
1                    Eastport  38.971501 -76.478293
2                        None        NaN        NaN
3                        None        NaN        NaN
4                 Waters Park  38.939740 -76.504025
5                        None        NaN        NaN
6                 West Street  38.980248 -76.525518
7                   Spa Creek  38.971490 -76.496312
8                   Bay Ridge  38.979575 -76.506042
9                        None        NaN        NaN
10                       None        NaN        NaN
11                       None        NaN        NaN
12                 Back Creek  38.965903 -76.479608
13              Severn Avenue  38.972013 -76.481889
14                       None        NaN        NaN
15                       None        NaN        NaN
16                       None        NaN        NaN
17                       None        NaN        NaN
18                   Maryland  38.986408 -76.500863
19                       None        NaN        NaN
20                       None        NaN        NaN
21                       None        NaN        NaN
22                       None        NaN        NaN
23           Annapolis Harbor  38.977292 -76.485563
24                       None        NaN        NaN
25         St. John's College  38.982283 -76.492238
26                      Acton  38.974574 -76.493253
27                       None        NaN        NaN
28                       None        NaN        NaN
29                       None        NaN        NaN
30              College Creek  38.985874 -76.489511
31                       None        NaN        NaN
32                       None        NaN        NaN
33                       None        NaN        NaN
34                       None        NaN        NaN
35                       None        NaN        NaN
36                       None        NaN        NaN
37                       None        NaN        NaN
38                       None        NaN        NaN
39                       None        NaN        NaN
40  West Annapolis Elementary  38.991625 -76.503391
41            Admiral Heights  38.984834 -76.518573
42                       None        NaN        NaN
43                Weems Creek  38.994568 -76.505458
44                Broad Creek        NaN        NaN
45                       None        NaN        NaN
46             Annapolis Mall  38.989885 -76.538600
47                       None        NaN        NaN
48                       None        NaN        NaN
49                       None        NaN        NaN

## Build an interactive Folium map

In [26]:
# Cell 5: Build an interactive Folium map

m = folium.Map(location=[38.9776, -76.4922], zoom_start=13)

for _, row in tweets_df.dropna(subset=['pred_lat']).iterrows():
    folium.CircleMarker(
        location=[row['pred_lat'], row['pred_lon']],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=row['tweet_text']
    ).add_to(m)

popup = (
    f"<b>toponym:</b> {row['toponym']}<br>"
    f"<b>Severity:</b> {row['severity']}<br>"
    #f"<b>Urgency:</b> {row['urgency']}<br>"
    f"<i>{row['tweet_text']}</i>"
)
folium.CircleMarker(
    location=[row['pred_lat'], row['pred_lon']],
    radius=4, color='red', fill=True, fill_opacity=0.7,
    popup=popup
).add_to(m)



# Save and display
m.save('flood_tweet_map.html')
m


## Spatial Clustering with DBSCAN

In [27]:
from sklearn.cluster import DBSCAN
import numpy as np

# 1) Prepare only rows with valid geocodes
valid = tweets_df.dropna(subset=['pred_lat','pred_lon']).copy()

# 2) Convert to radians for haversine metric
coords = np.radians(valid[['pred_lat','pred_lon']].values)

# 3) DBSCAN: eps=0.5 km neighborhood, min 5 points
db = DBSCAN(
    eps=0.5/6371.0088,       # 0.5 km in radians
    min_samples=5,
    metric='haversine',
    algorithm='ball_tree'
).fit(coords)

# 4) Attach cluster labels back
valid['cluster'] = db.labels_
tweets_df.loc[valid.index, 'dbscan_cluster'] = valid['cluster']


## Heat Map of All Reports

In [28]:
import folium
from folium.plugins import HeatMap

# Base map
m = folium.Map(location=[38.9776, -76.4922], zoom_start=13)

# Heat layer
heat_data = [
    [row['pred_lat'], row['pred_lon']]
    for _, row in tweets_df.dropna(subset=['pred_lat','pred_lon']).iterrows()
]
HeatMap(heat_data, radius=15, blur=25).add_to(m)

# Save & display
m.save('flood_tweet_heatmap.html')
m
